In [8]:
from scripts.neuron_signal_generator import generate_signal
from scripts.noise_and_filtering import generate_electrode_signal
from scripts.simulator import simulate_recording

from classes.sorter import SpikeSorter
import numpy as np
# from scripts.spike_extraction import get_waveform_data

In [2]:
# generate signal
neuron_signal = generate_signal()

generate_electrode_signal(
    signal=neuron_signal,
    decay_type='square',
    decay_rate=2,
    noise_type='gaussian',
    noise_std=0.5,
    filter_type='bandpass',
    low=500,
    high=3000
)

placements = [2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0]
neuronParams = {"neuron_type": "standard", "lambda": 14, "v_rest": -70, "v_thres": -10, "t_ref": 0.02, "fix_random_seed": True}
processingParams = {"decay_type": "square", "decay_rate": 2, "noise_type": "gaussian", "noise_std": 0.5, "filter_type": "none", "low": 500, "high": 3000}

time, filtered_signals = simulate_recording(placements, neuronParams, processingParams)

In [17]:
def get_waveform_data(signals, multiplier, waveform_duration):

    # intialise the spike sorter
    sorter = SpikeSorter(threshold_factor=multiplier, sample_rate=25000, waveform_duration=waveform_duration)

    # get the spikes for each recording
    spikes = sorter.get_spikes(signals)

    # convert the spikes into a numpy array
    spike_indices = np.array(spikes)

    # neuron spikes is used to determine the true labels of the spike data, so we'll just give an empty array
    neuron_spikes = []

    #### something not working here

    # get the locations of all the spikes detected by all channels, and true spike posititons
    merged_spike_indices, true_labels = sorter.merge_spike_indices(spike_indices, neuron_spikes, tolerance=30)

    print(merged_spike_indices)

    # sorter.show_spikes(signals, [merged_spike_indices] * len(signals), figsize=(8, 8))

    # extract the waveform data from each of the identified spikes across all electrodes
    waveforms, waveform_info = sorter.get_all_waveforms(
        signals, 
        merged_spike_indices, 
        recenter=True,
        # visualise=True,
        labels=true_labels 
    )

    return waveforms, waveform_info


In [24]:
data = {
    "signals": filtered_signals,
    "extractionParams": {
        "thresholdMultiplier": 4,
        "waveformDuration": 0.3
    }
}

signals = data["signals"]
extraction_data = data["extractionParams"]
multiplier, waveform_duration = extraction_data["thresholdMultiplier"], extraction_data["waveformDuration"]

waveforms, waveform_info = get_waveform_data(signals, multiplier, waveform_duration)

waveforms = waveforms.tolist()
waveform_info = waveform_info.tolist()

[10121 11082 17000 19890 21985 22946 23907]


In [29]:
type(waveform_info[0][0])

dict